In [29]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [30]:
X=[]
Z=[]
IMG_SIZE=150
YELLOW_DIR="./data/train/yellow"
GREEN_DIR="./data/train/green"
OPEN_DIR="./data/train/open"
UNKNOWN_DIR="./data/unknown"

In [31]:
def assign_label(img,plant_type):
    return plant_type

In [32]:
def make_train_data(plant_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,plant_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

In [33]:
make_train_data('Yellow',YELLOW_DIR)
print(len(X))

100%|██████████| 113/113 [00:00<00:00, 579.36it/s]

113


In [34]:
make_train_data('Green',GREEN_DIR)
print(len(X))

100%|██████████| 334/334 [00:00<00:00, 625.28it/s]

447


In [35]:
make_train_data('Open',OPEN_DIR)
print(len(X))

100%|██████████| 61/61 [00:00<00:00, 531.92it/s]

508


In [ ]:
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Plant: '+Z[l])
        
plt.tight_layout()

In [51]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,3)
X=np.array(X)
X=X/255
print (X.shape)
#print (X[2][0][0][0])
print (Y[500])

(508, 150, 150, 3)
[0. 1. 0.]


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [28]:
type(x_train)
x_train.shape

(381, 150, 150, 3)

In [ ]:
np.random.seed(42)
rn.seed(42)
tf.set_random_seed(42)

In [ ]:
# # modelling starts using a CNN.

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(3, activation = "softmax"))

In [ ]:
batch_size=128
epochs= 1500

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)
#es = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=0.01, restore_best_weights=True)
ckp = ModelCheckpoint("./cp/chkpt2.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift  images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)


In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#model.summary()

In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              callbacks=[red_lr, ckp],
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
#save weights and model
model.save_weights("./models/10epochmodel")
model_json = model.to_json()
with open("./models/10eph_model", "w") as json_file:
    json_file.write(model_json)

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
History.history

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# getting predictions on val set.
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

#pred2 = model.predict()


In [ ]:
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:
len(mis_class)

In [ ]:

fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
ax[0,0].imshow(x_test[prop_class[1]])
#ax[0,0].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[prop_class[0]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[prop_class[0]]]))))
([pred_digits[prop_class[1]]])
#le.inverse_transform([pred_digits[prop_class[1]]])
#le.inverse_transform([y_test[prop_class[1]]])
#ax[0,0].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[prop_class[0]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[prop_class[0]]]))))


In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[prop_class[count]]])))#+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[prop_class[count]]]))))
        #plt.tight_layout()
        count+=1

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (1):
    for j in range (1):
        ax[i,j].imshow(x_test[mis_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[mis_class[count]]])))#+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[mis_class[count]]]))))
        plt.tight_layout()
        count+=1

In [ ]:
S=[]
T=[]
F=[]

In [ ]:
def make_unk_data(plant_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,plant_type)
        img2 = img
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        F.append(img2)
        S.append(np.array(img)) #like X
        T.append(str(label)) #like Z

In [ ]:
UNKNOWN_DIR="./data/unknown"
make_unk_data('Unknown',UNKNOWN_DIR)

In [ ]:
def make_unk_data(plant_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,plant_type)
        img2 = img
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        F.append(img2)
        S.append(np.array(img)) #like X
        T.append(str(label)) #like Z

In [ ]:
le2=LabelEncoder()
R=le.fit_transform(T) #like Y
Y=to_categorical(R,3)
S=np.array(S)
S=S/255

In [ ]:


pred2 = model.predict(S)
print("green, open, yellow")
#print (pred2)
list = pred2.tolist()
for i in range(len(list)):
    print(list[i], " ", F[i])

